In [4]:
!pip install pandas

     --------------------------------------- 10.6/10.6 MB 97.1 kB/s eta 0:00:00
     ------------------------------------- 503.5/503.5 KB 99.0 kB/s eta 0:00:00
     --------------------------------------- 14.7/14.7 MB 83.6 kB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import re

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df_ = pd.read_csv('tweet_labeled.csv')
df_

In [ ]:
df = df_[["date" , "text", "label_1D" ]]
df

In [2]:
tweets = df["text"].tolist()
tweets[0:11]

NameError: name 'df' is not defined

In [3]:
def processTweet(tweet): #start process_tweet
    # process the tweets
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)#trim
    tweet = tweet.strip('\'"')
    return tweet
#end
tweets_cleaned = []
#Read the tweets one by one and process it
for i in range (0,len(tweets)):
    processedTweet = processTweet(tweets[i])
    tweets_cleaned.append(processedTweet)
print(len(tweets_cleaned))
tweets_cleaned[0:100]

NameError: name 'tweets' is not defined

In [7]:
df.assign(text=tweets_cleaned)

,date,text,label_1D
0,2021-02-05 10:52:00,2⃣ debunking 9 bitcoin myths by AT_USER ⬇️ cry...,0.068038
1,2021-02-05 10:52:00,📖 weekend read 📖 keen to learn about crypto as...,0.068038
2,2021-02-05 10:52:00,4⃣ 🎙️ bloomberg lp cryptooutlook 2021 with AT_...,0.068038
3,2021-02-05 10:52:00,reddcoin rdd AT_USER to the moon altcoin turnr...,0.068038
4,2021-02-05 10:52:00,"5⃣ blockchain 50 2021 by AT_USER AT_USER , AT_...",0.068038
...,...,...,...
2284908,2022-02-09 23:59:00,any recommendations for a site that can do wha...,-0.019778
2284909,2022-02-09 23:59:00,AT_USER running out of bitcoin,-0.019778
2284910,2022-02-09 23:59:00,i wish you success in this valuable campaign. ...,-0.019778
2284911,2022-02-09 23:59:00,wish i had more money for crypto... bitcoin,-0.019778


In [16]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pandas.core.frame import DataFrame

vader = SentimentIntensityAnalyzer()
def getCoumpoundScore(Tweets):
    pos = vader.polarity_scores(Tweets)['pos']
    neg = vader.polarity_scores(Tweets)['neg']
    neu = vader.polarity_scores(Tweets)['neu']
    #score = vader.polarity_scores(Tweets)['compound']
    return pos, neg, neu 
pos = []
neg = []
neu = []
#Read the tweets one by one and get the sentiment score
for i in range (0,len(tweets_cleaned)):
    tweets_pos, tweets_neg, tweets_neu = getCoumpoundScore(tweets_cleaned[i])
    pos.append(tweets_pos)
    neg.append(tweets_neg)
    neu.append(tweets_neu)
#store the tweet with polarity score in a dictionary
df_scores ={"tweets": tweets_cleaned, "positive" : pos, "negative" : neg, "neutral" : neu}
df_scores = DataFrame(df_scores)#transfer into a dataframe
print(df_scores.shape)
df_scores.head(11)

(2284913, 4)


,tweets,positive,negative,neutral
0,2⃣ debunking 9 bitcoin myths by AT_USER ⬇️ cry...,0.000,0.0,1.000
1,📖 weekend read 📖 keen to learn about crypto as...,0.183,0.0,0.817
2,4⃣ 🎙️ bloomberg lp cryptooutlook 2021 with AT_...,0.000,0.0,1.000
3,reddcoin rdd AT_USER to the moon altcoin turnr...,0.000,0.0,1.000
4,"5⃣ blockchain 50 2021 by AT_USER AT_USER , AT_...",0.000,0.0,1.000
5,bitcoin and eth both have bullish setups for a...,0.151,0.0,0.849
6,$perl 0.06. i have insisted that since 0.02 it...,0.091,0.0,0.909
7,amazing 😍 monopoly crypto cryptocurrency crypt...,0.362,0.0,0.638
8,AT_USER $juld $bnb binance bsc binancesmartcha...,0.000,0.0,1.000
9,amazoncoin = 100% love 💚 👉👉 URL $ama amazoncoi...,0.307,0.0,0.693


In [17]:
merged_data = pd.concat([df,df_scores],axis=1).reset_index(drop=True)
merged_data = merged_data.drop('text',axis=1)
print(merged_data.shape)
merged_data.head(10)

(2284913, 6)


,date,label_1D,tweets,positive,negative,neutral
0,2021-02-05 10:52:00,0.068038,2⃣ debunking 9 bitcoin myths by AT_USER ⬇️ cry...,0.000,0.0,1.000
1,2021-02-05 10:52:00,0.068038,📖 weekend read 📖 keen to learn about crypto as...,0.183,0.0,0.817
2,2021-02-05 10:52:00,0.068038,4⃣ 🎙️ bloomberg lp cryptooutlook 2021 with AT_...,0.000,0.0,1.000
3,2021-02-05 10:52:00,0.068038,reddcoin rdd AT_USER to the moon altcoin turnr...,0.000,0.0,1.000
4,2021-02-05 10:52:00,0.068038,"5⃣ blockchain 50 2021 by AT_USER AT_USER , AT_...",0.000,0.0,1.000
5,2021-02-05 10:53:00,0.069270,bitcoin and eth both have bullish setups for a...,0.151,0.0,0.849
6,2021-02-05 10:54:00,0.067211,$perl 0.06. i have insisted that since 0.02 it...,0.091,0.0,0.909
7,2021-02-05 10:57:00,0.068002,amazing 😍 monopoly crypto cryptocurrency crypt...,0.362,0.0,0.638
8,2021-02-05 10:58:00,0.068632,AT_USER $juld $bnb binance bsc binancesmartcha...,0.000,0.0,1.000
9,2021-02-05 10:58:00,0.068632,amazoncoin = 100% love 💚 👉👉 URL $ama amazoncoi...,0.307,0.0,0.693


In [19]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(merged_data[['positive', 'negative', 'neutral']],merged_data['label_1D'], test_size=0.005, train_size=0.025, shuffle=True )

In [25]:
 X_train.shape[1]

3

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

dim = X_train.shape[1]
# define our MLP network
model = Sequential()
model.add(Dense(8, input_dim=dim, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="linear"))

In [50]:
#def custom_MAE (y_true, y_pred):
#    
#    df_[MAE] = np.where((np.sign(df_[predict])) == np.sign(df_[label]) , 20*mean_absolute_error(df_[label],df_[predict])
#                       , 60*mean_absolute_error(df_[label],df_[predict]))
#    result = df_[MAE].mean()               
#    return result

def custom_MAE(y_true, y_pred):
 
    loss = np.where((np.sign(y_pred)) == np.sign(y_true) , 20*mean_absolute_error(y_true, y_pred)
                       , 60*mean_absolute_error(y_true, y_pred))    
    result = loss.mean()       
    
    return result

In [51]:
custom_MAE(y_test, preds)

C:\Users\robin\AppData\Local\Temp/ipykernel_12420/2137811338.py:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  loss = np.where((np.sign(y_pred)) == np.sign(y_true) , 20*mean_absolute_error(y_true, y_pred)


ValueError: Data must be 1-dimensional

In [47]:
from tensorflow.keras.optimizers import Adam

opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="custom_MAE", optimizer=opt)

c:\users\robin\appdata\local\programs\python\python38\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 32        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 73
Trainable params: 73
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),
    epochs=200, batch_size=10)

Epoch 1/200
5713/5713 [==============================] - 22s 3ms/step - loss: 138.1213 - val_loss: 128.3471
Epoch 2/200
5713/5713 [==============================] - 11s 2ms/step - loss: 141.2204 - val_loss: 113.6118
Epoch 3/200
5713/5713 [==============================] - 13s 2ms/step - loss: 135.7356 - val_loss: 101.4569
Epoch 4/200
5713/5713 [==============================] - 11s 2ms/step - loss: 137.9907 - val_loss: 103.9696
Epoch 5/200
5713/5713 [==============================] - 13s 2ms/step - loss: 136.9614 - val_loss: 108.1653
Epoch 6/200
5713/5713 [==============================] - 10s 2ms/step - loss: 134.0988 - val_loss: 106.2270
Epoch 7/200
5713/5713 [==============================] - 10s 2ms/step - loss: 132.5190 - val_loss: 116.8002
Epoch 8/200
5713/5713 [==============================] - 12s 2ms/step - loss: 133.7870 - val_loss: 252.3346
Epoch 9/200
5713/5713 [==============================] - 10s 2ms/step - loss: 135.8148 - val_loss: 245.7822
Epoch 10/200
5713/5713 [====

5713/5713 [==============================] - 9s 2ms/step - loss: 106.8546 - val_loss: 100.4219
Epoch 152/200
5713/5713 [==============================] - 9s 2ms/step - loss: 106.2451 - val_loss: 194.2187
Epoch 153/200
5713/5713 [==============================] - 9s 2ms/step - loss: 106.9382 - val_loss: 103.5807
Epoch 154/200
5713/5713 [==============================] - 9s 2ms/step - loss: 107.1846 - val_loss: 104.4560
Epoch 155/200
5713/5713 [==============================] - 9s 2ms/step - loss: 107.6365 - val_loss: 103.3406
Epoch 156/200
5713/5713 [==============================] - 9s 2ms/step - loss: 106.8400 - val_loss: 114.2971
Epoch 157/200
5713/5713 [==============================] - 11s 2ms/step - loss: 107.1531 - val_loss: 100.1961
Epoch 158/200
5713/5713 [==============================] - 10s 2ms/step - loss: 107.1517 - val_loss: 102.1361
Epoch 159/200
5713/5713 [==============================] - 11s 2ms/step - loss: 106.6150 - val_loss: 125.2696
Epoch 160/200
5713/5713 [=====

In [59]:
preds = model.predict(X_test)
preds.shape

(11425, 1)

In [58]:
np.array(y_test).shape


(11425,)

In [43]:
import locale

# make predictions on the testing data
preds = model.predict(X_test)
 
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - y_test
percentDiff = (diff / y_test) * 100
absPercentDiff = np.abs(percentDiff)
 
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
 
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("avg. : {}, std : {}".format(
    locale.currency(df["label_1D"].mean(), grouping=True),
    locale.currency(df["label_1D"].std(), grouping=True)))
print("mean: {:.2f}%, std: {:.2f}%".format(mean, std))

avg. : $0.00, std : $0.04
mean: 101.79%, std: 67.34%


In [ ]:
custom_MAE()